In [1]:
import tweepy
import pandas as pd
import nltk
import string
import os
import re
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
consumer_key = "Ax79DI6gXdMjOCQyQ61v2Khbj"
consumer_secret = "DBSmfUL50FlhpVnePa373lOpBFffJjrGInXbSGG4T7Ze2RRQTl"
access_token = "1418173405988376579-9g3SF5klnWOgd62nUfTfaqrQdzkJRA"
access_token_secret = "q3jO28xbGDlvE6djCNeoSMQbtkosZYT7mBTv0O6wznMpw"
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


In [2]:
spark = SparkSession.builder.appName('logistic').getOrCreate()
df = spark.read.csv("trainingandtestdata/training.1600000.processed.noemoticon.csv")
df = df.withColumnRenamed("_c0", 'polarity')
df = df.withColumnRenamed("_c1", 'tweet_id')
df = df.withColumnRenamed("_c2", 'tweet_date')
df = df.withColumnRenamed("_c3", 'query_string')
df = df.withColumnRenamed("_c4", 'tweet_screen_name')
df = df.withColumnRenamed("_c5", 'text')
df.show(5)

+--------+----------+--------------------+------------+-----------------+--------------------+
|polarity|  tweet_id|          tweet_date|query_string|tweet_screen_name|                text|
+--------+----------+--------------------+------------+-----------------+--------------------+
|       0|1467810369|Mon Apr 06 22:19:...|    NO_QUERY|  _TheSpecialOne_|@switchfoot http:...|
|       0|1467810672|Mon Apr 06 22:19:...|    NO_QUERY|    scotthamilton|is upset that he ...|
|       0|1467810917|Mon Apr 06 22:19:...|    NO_QUERY|         mattycus|@Kenichan I dived...|
|       0|1467811184|Mon Apr 06 22:19:...|    NO_QUERY|          ElleCTF|my whole body fee...|
|       0|1467811193|Mon Apr 06 22:19:...|    NO_QUERY|           Karoli|@nationwideclass ...|
+--------+----------+--------------------+------------+-----------------+--------------------+
only showing top 5 rows



In [3]:
df=df[['polarity','text']]
df=df.withColumn('pre_clean_len',F.length('text'))
df.show(5)

+--------+--------------------+-------------+
|polarity|                text|pre_clean_len|
+--------+--------------------+-------------+
|       0|@switchfoot http:...|          115|
|       0|is upset that he ...|          111|
|       0|@Kenichan I dived...|           89|
|       0|my whole body fee...|           47|
|       0|@nationwideclass ...|          111|
+--------+--------------------+-------------+
only showing top 5 rows



In [4]:
# Follow every follower of the authenticated user
timeline = api.home_timeline()
data = pd.DataFrame(data=[tweet.text for tweet in timeline], columns=['text'])